In [1]:
from nose.tools import assert_equal

In [65]:
CSV_PATH = "2001.csv"

In [66]:
!ln -s $CSV_PATH 2001.csv

ln: 2001.csv: File exists


In [70]:
%%writefile airport.sql

DROP TABLE IF EXISTS flights;

CREATE TABLE flights (
    year INT,
    month INT,
    dayOfMonth INT,
    dayOfWeek INT,
    actualDepartureTime INT,
    scheduledDepartureTime INT,
    arrivalArrivalTime INT,
    scheduledArrivalTime INT,
    uniqueCarrierCode TEXT,
    flightNumber INT,
    tailNumber TEXT,
    actualElapsedTime INT,
    scheduledElapsedTime INT,
    airTime INT,
    arrivalDelay INT,
    departureDelay INT,
    originCode TEXT,
    destinationCode TEXT,
    distance INT,
    taxiIn INT,
    taxiOut INT,
    cancelled INT,
    cancellationCode TEXT,
    diverted INT,
    carrierDelay INT,
    weatherDelay INT,
    nasDelay INT,
    securityDelay INT,
    lateAircraftDelay INT
);

.separator ,
.import 2001.csv flights

-- Our file has a header. The following line deletes the header.
DELETE FROM flights WHERE Year='Year';

Writing airport.sql


In [71]:
!sqlite3 assignment.db < airport.sql

## Problem 1: Creating a Database

### 1.1 Count flights table

In [76]:
%%writefile count_lines_flights.sql
SELECT COUNT(*) FROM flights;

Writing count_lines_flights.sql


In [77]:
nlines_flights = !sqlite3 assignment.db < count_lines_flights.sql
print(nlines_flights.s)
assert_equal(nlines_flights.s, "5967780")

5967780


### 1.2 Creating another table

In [81]:
!curl -s https://raw.githubusercontent.com/UI-DataScience/info490-fa15/master/Week12/assignment/iata.csv  -o iata.csv 

In [86]:
%%writefile import_iata.sql

DROP TABLE iata ;
    
CREATE TABLE iata (
    airportID INT NOT NULL,
    name TEXT,
    city TEXT,
    country TEXT,
    iata TEXT,
    icao TEXT, 
    latitude REAL, 
    longitude REAL, 
    altitude INT, 
    timeZone INT, 
    dst TEXT, 
    tzDatabaseTimeZone TEXT
) ;

.mode csv

.import iata.csv iata

Overwriting import_iata.sql


In [87]:
!sqlite3 assignment.db < import_iata.sql

In [89]:
!sqlite3 assignment.db "SELECT * FROM iata LIMIT 10"

1|Goroka|Goroka|Papua New Guinea|GKA|AYGA|-6.081689|145.391881|5282|10|U|Pacific/Port_Moresby
2|Madang|Madang|Papua New Guinea|MAG|AYMD|-5.207083|145.7887|20|10|U|Pacific/Port_Moresby
3|Mount Hagen|Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789|144.295861|5388|10|U|Pacific/Port_Moresby
4|Nadzab|Nadzab|Papua New Guinea|LAE|AYNZ|-6.569828|146.726242|239|10|U|Pacific/Port_Moresby
5|Port Moresby Jacksons Intl|Port Moresby|Papua New Guinea|POM|AYPY|-9.443383|147.22005|146|10|U|Pacific/Port_Moresby
6|Wewak Intl|Wewak|Papua New Guinea|WWK|AYWK|-3.583828|143.669186|19|10|U|Pacific/Port_Moresby
7|Narsarsuaq|Narssarssuaq|Greenland|UAK|BGBW|61.160517|-45.425978|112|-3|E|America/Godthab
8|Nuuk|Godthaab|Greenland|GOH|BGGH|64.190922|-51.678064|283|-3|E|America/Godthab
9|Sondre Stromfjord|Sondrestrom|Greenland|SFJ|BGSF|67.016969|-50.689325|165|-3|E|America/Godthab
10|Thule Air Base|Thule|Greenland|THU|BGTL|76.531203|-68.703161|251|-4|E|America/Thule


In [90]:
iata_exists = !sqlite3 assignment.db "SELECT name FROM sqlite_master WHERE type='table' and name='iata'"
assert_equal(iata_exists.s, "iata")

In [91]:
iata_info = !sqlite3 assignment.db "PRAGMA table_info(iata)"
iata_names = [i.split("|")[1] for i in iata_info]
iata_names_answer = [
    "airportID",
    "name",
    "city",
    "country",
    "iata",
    "icao",
    "latitude",
    "longitude",
    "altitude",
    "timeZone",
    "dst",
    "tzDatabaseTimeZone"
]
assert_equal(len(iata_names), len(iata_names_answer))
assert_equal(set(iata_names), set(iata_names_answer))

### 1.3. Count iata table

In [92]:
%%writefile count_lines_iata.sql
SELECT COUNT(*) FROM iata;

Writing count_lines_iata.sql


In [93]:
nlines_iata = !sqlite3 assignment.db < count_lines_iata.sql
print(nlines_iata.s)

assert_equal(nlines_iata.s, "8107")

8107


## Problem 2. Data Manipulation

### 2.1 Joining tables

In [3]:
%%writefile join.sql

CREATE TABLE myTable AS
  SELECT f.month, f.dayOfMonth, f.uniqueCarrierCode, f.flightNumber, f.scheduledDepartureTime, 
         f.diverted, i.city
    FROM flights AS f, iata AS i
    where f.destinationCode=i.iata;
  

Overwriting join.sql


In [4]:
!sqlite3 assignment.db < join.sql

In [5]:
!sqlite3 assignment.db "SELECT * FROM myTable LIMIT 10;"

1|17|US|375|1810|0|Charlotte
1|18|US|375|1810|0|Charlotte
1|19|US|375|1810|0|Charlotte
1|20|US|375|1810|0|Charlotte
1|21|US|375|1810|0|Charlotte
1|22|US|375|1810|0|Charlotte
1|23|US|375|1810|0|Charlotte
1|24|US|375|1810|0|Charlotte
1|25|US|375|1810|0|Charlotte
1|26|US|375|1810|0|Charlotte


In [8]:
mytable_exists = !sqlite3 assignment.db "SELECT name FROM sqlite_master WHERE type='table' AND name='myTable';"
assert_equal(mytable_exists.s, "myTable")

table_info = !sqlite3 assignment.db "PRAGMA table_info(myTable);"

mytable_names = [i.split("|")[1] for i in table_info]
mytable_names_answer = [
    'month', 'dayOfMonth', 'uniqueCarrierCode', 'flightNumber', 'scheduledDepartureTime', 'diverted', 'city'
]
assert_equal(len(mytable_names), len(mytable_names_answer))
assert_equal(set(mytable_names), set(mytable_names_answer))

# do some sample queries
mytable_count_ord = !sqlite3 assignment.db "SELECT COUNT(*) FROM myTable WHERE city='Chicago';"
assert_equal(mytable_count_ord.s, "391776")

mytable_count_sfo = !sqlite3 assignment.db "SELECT COUNT(*) FROM myTable WHERE city='San Francisco';"
assert_equal(mytable_count_sfo.s, "121738")

### 2.2 Inserting

In [12]:
%%writefile insert.sql
INSERT INTO myTable 
VALUES(9, 9, 'INFO',490, 0800, 1, 'Champaign') ;

Overwriting insert.sql


In [14]:
!sqlite3 assignment.db <insert.sql

In [15]:
info_month = !sqlite3 assignment.db "SELECT month FROM myTable WHERE uniqueCarrierCode='INFO';"
info_day = !sqlite3 assignment.db "SELECT dayOfMonth FROM myTable WHERE uniqueCarrierCode='INFO';"
info_flight_no = !sqlite3 assignment.db "SELECT flightNumber FROM myTable WHERE uniqueCarrierCode='INFO';"
info_crs_dep = !sqlite3 assignment.db "SELECT scheduledDepartureTime FROM myTable WHERE uniqueCarrierCode='INFO';"
info_diverted = !sqlite3 assignment.db "SELECT diverted FROM myTable WHERE uniqueCarrierCode='INFO';"
info_dest = !sqlite3 assignment.db "SELECT city FROM myTable WHERE uniqueCarrierCode='INFO';"

print('''
UniqueCarrierCode: {0}
Month: {1}
Day: {2}
Flight Number: {3}
Scheduled Departure Time: {4}
Diverted: {5}
Origin City: {6}
'''.format(
    "INFO",
    info_month.s,
    info_day.s,
    info_flight_no.s,
    info_crs_dep.s,
    info_diverted.s,
    info_dest.s
    )
)


UniqueCarrierCode: INFO
Month: 9
Day: 9
Flight Number: 490
Scheduled Departure Time: 800
Diverted: 1
Origin City: Champaign



In [16]:
assert_equal("9", info_month.s)
assert_equal("9", info_day.s)
assert_equal("490", info_flight_no.s)
assert_equal("800", info_crs_dep.s)
assert_equal("1", info_diverted.s)
assert_equal("Champaign", info_dest.s)

## Problem 3. Query

### 3.1 Average

In [27]:
%%writefile get_average_depdelay.sql
SELECT AVG(departureDelay) FROM flights
WHERE departureDelay <> 'NA';

Overwriting get_average_depdelay.sql


In [29]:
average_depdelay = !sqlite3 assignment.db < get_average_depdelay.sql
print(average_depdelay)
assert_equal(average_depdelay.s, '8.15483749731112')

['8.15483749731112']


### 3.2 Maximum

In [24]:
%%writefile get_maximum_depdelay.sql
SELECT MAX(departureDelay) FROM flights
WHERE departureDelay <> 'NA';

Overwriting get_maximum_depdelay.sql


In [26]:
maximum_depdelay = !sqlite3 assignment.db < get_maximum_depdelay.sql
print(maximum_depdelay)
assert_equal(maximum_depdelay.s, '1692')

['1692']


### 3.3 Minimum

In [30]:
%%writefile get_minimum_depdelay.sql
SELECT MIN(departureDelay) FROM flights
WHERE departureDelay <> 'NA';

Writing get_minimum_depdelay.sql


In [31]:
minimum_depdelay = !sqlite3 assignment.db < get_minimum_depdelay.sql
print(minimum_depdelay)
assert_equal(minimum_depdelay.s, '-204')

['-204']


### 3.4 Search

In [38]:
%%writefile find.sql
SELECT Month, dayOfMonth, uniqueCarrierCode, flightNumber FROM myTable
WHERE Month=9 AND scheduledDepartureTime>745 AND scheduledDepartureTime<815 AND city='San Francisco' AND diverted=1;

Overwriting find.sql


In [39]:
query = !sqlite3 assignment.db < find.sql
print(query)

['9|11|UA|93', '9|11|UA|301', '9|11|UA|805', '9|11|UA|809', '9|22|UA|2377', '9|11|US|63', '9|11|AA|59']


In [40]:
query_answer = ['9|11|UA|93', '9|11|UA|301', '9|11|UA|805', '9|11|UA|809', '9|22|UA|2377', '9|11|US|63', '9|11|AA|59']

assert_equal(len(query), len(query_answer))
assert_equal(set(query), set(query_answer))